In [1]:
import pandas as pd
import numpy as np
import csv
import MeCab
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import re
import neologdn
import demoji
import emoji

# model_dir = '/Users/iomacbookair2/Documents/lab/DEIM2023/entity_vector/entity_vector.model.bin'
model_dir = '/Users/labimac/Documents/lab/DEIM2023/entity_vector/entity_vector.model.bin'
model_word2vec = KeyedVectors.load_word2vec_format(model_dir, binary=True)

In [2]:
def calc_similarity_word2vec(sentence1, sentence2, model):
  mecab = MeCab.Tagger(
      '-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')

  pre_sentence1 = sentence1.split(" ")
  sentence1_words = []
  for s1 in pre_sentence1:
    sentence1_words += [line.split("\t")[0]
                        for line in mecab.parse(s1).split("\n")[:-2]]
  # print(f"sentence1_words: {sentence1_words}")

  pre_sentence2 = sentence2.split(" ")
  sentence2_words = []
  for s2 in pre_sentence2:
    sentence2_words += [line.split("\t")[0]
                        for line in mecab.parse(s2).split("\n")[:-2]]
  # print(f"sentence2_words: {sentence2_words}")

  if not sentence1_words and not sentence2_words:
    similarity = 1
  elif not sentence1_words or not sentence2_words:
    similarity = 0
  elif (len(sentence1_words) == 1 and len(sentence2_words) != 1) or (len(sentence2_words) == 1 and len(sentence1_words) != 1):
    similarity = 0
  elif all(re.match(r'.*[a-zA-Z].*', word) for word in sentence1_words) or all(re.match(r'.*[a-zA-Z].*', word) for word in sentence2_words):
    similarity = 0
  else:
    # Compute word embeddings for each sentence
    sentence1_embedding = np.mean([model[word]
                                  for word in sentence1_words if word in model], axis=0)
    # print(f"sentence1_embedding: {sentence1_embedding}")
    sentence2_embedding = np.mean([model[word]
                                  for word in sentence2_words if word in model], axis=0)
    # print(f"sentence2_embedding: {sentence2_embedding}")
    if np.isnan(sentence1_embedding).any() or np.isnan(sentence2_embedding).any():
        similarity = 0
    else:
      similarity = cosine_similarity(
          [sentence1_embedding], [sentence2_embedding])[0][0]

  # print(f"similarity: {similarity}")

  return similarity


def calc_average_similarity_word2vec(sentences, key):
    similarities = []
    for i in range(len(sentences)):
        for j in range(i + 1, len(sentences)):
            similarity = calc_similarity_word2vec(
                sentences[i], sentences[j], model_word2vec)
            similarities.append(similarity)

    q = [0, 0.25, 0.5, 0.75, 1]
    outputs = {}
    for i in range(len(q)):
        outputs[f"quantile_{i}"] = np.quantile(similarities, q[i])
    outputs["average_similarity"] = np.mean(similarities)
    outputs["size"] = len(list(np.sort(np.array(similarities))))
    outputs["width_q"] = outputs["quantile_3"] - outputs["quantile_1"]
    outputs["variance"] = np.var(similarities)

    if key == "quantile_3":
        return outputs["quantile_3"]
    elif key == "average":
        return outputs["average_similarity"]
    elif key == "size":
        return outputs["size"]
    elif key == "width":
        return outputs["width_q"]
    elif key == "variance":
        return outputs["variance"]


In [3]:
# path = "/Users/iomacbookair2/Documents/lab/DEIM2023/tweet_csv/221213_ann_tue.csv"
path = "/Users/labimac/Documents/lab/DEIM2023/tweet_csv/221213_ann_tue.csv"
df = pd.read_csv((path))
df.sort_values(by = 'created_at', ascending = True, inplace = True)
df = df.reset_index(drop=True)
df['created_at'] = pd.to_datetime(df['created_at'])
df = df.drop("author_id", axis=1)
df = df.drop("username", axis=1)

df.to_csv("sorted.csv", index=False)
df

,created_at,text
0,2022-12-14 01:00:00,#星野源ANN
1,2022-12-14 01:00:00,#星野源ANN
2,2022-12-14 01:00:00,#星野源ANN
3,2022-12-14 01:00:00,#星野源ANN
4,2022-12-14 01:00:00,#星野源ANN
...,...,...
3343,2022-12-14 03:03:54,年末かぁー。 これからいろいろお楽しみあるから、あっという間に1月よね♪♪ 土井さん楽しみだ...
3344,2022-12-14 03:03:59,#星野源ANN 国性のハガキって手元に何枚か来てるんやね。
3345,2022-12-14 03:04:28,お疲れさま( ^^) _U~~ 今夜もありがとうございました♡ #星野源ANN
3346,2022-12-14 03:04:33,残HP0.2！！！！おうち帰って充電してくれ！！！🥺 #星野源ANN


In [4]:
def preprocess(text):
    text = emoji.replace_emoji(text, replace=' ')
    text = neologdn.normalize(text)
    text = re.sub(r'#\S+', '', text)
    text = re.sub(r'＃\S+', '', text)
    text = re.sub(r'http?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)
    text = re.sub(r'https?://\S+', ' ', text)
    text = re.sub(r'[!-/:-@[-`{-~]', r' ', text)
    text = re.sub(
        u'[■-♯【】「」『』・ㅂﾟˊᗜ、。∀〇╰ˋω…╭´｀•˘д↑艸╯→°д̀ᴗ˃˂⁽⁾φ＼※彡𖥦←ꂹ]', ' ', text)
    text = re.sub(r'(\d)([,.])(\d+)', r'\1\3', text)
    text = re.sub(r'\d+', '0', text)
    text = text.lower()
    text = re.sub(r"[\u3000\t\r\n]", " ", text)
    return text


df['text'] = df['text'].apply(preprocess)
df
df.to_csv('filtered3.csv', index=False)


In [5]:
groups = df.groupby(pd.Grouper(key='created_at', freq='min'))
df_texts_by_minute = pd.DataFrame({
    "texts_by_minute": groups.apply(lambda x: x["text"].tolist())
})
df_texts_by_minute = df_texts_by_minute.reset_index()
df_texts_by_minute
# df_texts_by_minute.to_csv("texts_by_minute.csv", index=False)

,created_at,texts_by_minute
0,2022-12-14 01:00:00,"[, , , , , 星野源のオールナイトニッポン ニッポン放送 0 0 0 火0 0 0..."
1,2022-12-14 01:01:00,"[ , , , リハーサル , お疲れなんですね , , 源さんこんばんは , , ,..."
2,2022-12-14 01:02:00,"[リハか お疲れ様です , イケボ , i m tiredおおお お疲れ様です, こ..."
3,2022-12-14 01:03:00,"[ほんまに源さんお疲れなのねでも がんばれぇ落ち着いてね , キス事変www , いいテンシ..."
4,2022-12-14 01:04:00,"[リハざんまいからのann お疲れモードの源さん アンニュイな感じもいいと思います, とある..."
...,...,...
120,2022-12-14 03:00:00,"[エンディング, アワード行きが多すぎなラジオ最高wwリハもお忙しそうだし hp0 0になる..."
121,2022-12-14 03:01:00,"[お疲れ様でしたhp0 0なのにがんばってくれてありがとうございました , お疲れ様でした..."
122,2022-12-14 03:02:00,"[hp0 0だけど最後までやれて良かった おうち帰ってフル回復してね , お疲れさまでした..."
123,2022-12-14 03:03:00,[お疲れ様でした 今日はお疲れ源さんでしたが こういうトーンの声は最高に好きです 来週は土井...


In [6]:
df_texts_by_minute["median"] = df_texts_by_minute["texts_by_minute"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity_word2vec(x, "quantile_3"))


/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric

In [7]:
df_texts_by_minute["average"] = df_texts_by_minute["texts_by_minute"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity_word2vec(x, "average"))


/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric

In [8]:
df_texts_by_minute["size"] = df_texts_by_minute["texts_by_minute"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity_word2vec(x, "size"))


/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric

In [9]:
df_texts_by_minute["width"] = df_texts_by_minute["texts_by_minute"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity_word2vec(x, "width"))


/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric

In [10]:
df_texts_by_minute["variance"] = df_texts_by_minute["texts_by_minute"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity_word2vec(x, "variance"))


/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric

In [11]:
df_texts_by_minute

,created_at,texts_by_minute,median,average,size,width,variance
0,2022-12-14 01:00:00,"[, , , , , 星野源のオールナイトニッポン ニッポン放送 0 0 0 火0 0 0...",1.000000,0.667899,20301,1.000000,0.217316
1,2022-12-14 01:01:00,"[ , , , リハーサル , お疲れなんですね , , 源さんこんばんは , , ,...",1.000000,0.341126,6903,1.000000,0.195341
2,2022-12-14 01:02:00,"[リハか お疲れ様です , イケボ , i m tiredおおお お疲れ様です, こ...",0.444460,0.232784,1891,0.444460,0.083605
3,2022-12-14 01:03:00,"[ほんまに源さんお疲れなのねでも がんばれぇ落ち着いてね , キス事変www , いいテンシ...",0.575540,0.344060,1891,0.575540,0.080646
4,2022-12-14 01:04:00,"[リハざんまいからのann お疲れモードの源さん アンニュイな感じもいいと思います, とある...",0.619481,0.373039,2278,0.619481,0.085895
...,...,...,...,...,...,...,...
120,2022-12-14 03:00:00,"[エンディング, アワード行きが多すぎなラジオ最高wwリハもお忙しそうだし hp0 0になる...",0.779716,0.524603,990,0.541426,0.101333
121,2022-12-14 03:01:00,"[お疲れ様でしたhp0 0なのにがんばってくれてありがとうございました , お疲れ様でした...",0.802628,0.544734,351,0.802628,0.116576
122,2022-12-14 03:02:00,"[hp0 0だけど最後までやれて良かった おうち帰ってフル回復してね , お疲れさまでした...",0.833680,0.762635,120,0.129990,0.011800
123,2022-12-14 03:03:00,[お疲れ様でした 今日はお疲れ源さんでしたが こういうトーンの声は最高に好きです 来週は土井...,0.868218,0.637365,66,0.326630,0.096646


In [12]:
df_texts_by_minute.to_csv('cosine_similarity_w2v_221213_ann_tue.csv', index=False)